In [5]:
using Plots
gr()

INFO: Recompiling stale cache file /Users/tlienart/.julia/lib/v0.6/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /Users/tlienart/.julia/lib/v0.6/PlotThemes.ji for module PlotThemes.
INFO: Recompiling stale cache file /Users/tlienart/.julia/lib/v0.6/Showoff.ji for module Showoff.
INFO: Recompiling stale cache file /Users/tlienart/.julia/lib/v0.6/FileIO.ji for module FileIO.
INFO: Precompiling module Requires.


Plots.GRBackend()

In [42]:
function compareGaussian(sxx, syy, mu, S, iS, d1=1, d2=2)
    
    Ss = S[[d1,d2],[d1,d2]]

    mu1, mu2 = mu[d1], mu[d2]
    s1,  s2  = sqrt(S[d1,d1]), sqrt(S[d2,d2])
    
    npts     = 250

    x1min, x1max = mu1 + 2*[-s1,s1]
    x2min, x2max = mu2 + 2*[-s2,s2]

    ng = 50 
    x  = linspace(x1min, x1max, ng)
    y  = linspace(x2min, x2max, ng)

    xgrid = repmat(x', ng, 1)
    ygrid = repmat(y, 1, ng)

    z = zeros(ng,ng)
    for i in 1:ng, j in 1:ng
        dx     = [x[i],y[j]]-[mu1,mu2]
        z[i,j] = exp(-dot(iS*dx, dx))
    end

 #   (xgrid, ygrid, z)
    contour(x,y,z)
    scatter!(sxx,syy, markersize=0.1)
end

compareGaussian(xx, yy, mu, S, iS)
#contour(xgrid, ygrid, z)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#fb9c07; stroke-width:1; stroke-opacity:1; fill:none" points="
 238.315,212.428 233.913,215.654 233.828,215.72 229.655,219.122 229.342,219.396 225.82,222.59 224.856,223.546 222.369,226.057 220.369,228.306 219.288,229.525 
 216.816,232.993 215.883,234.914 215.131,236.461 215.511,239.928 215.883,240.216 220.369,240.509 222.37,239.928 224.856,239.207 229.342,237.296 230.919,236.461 
 233.828,234.915 237.078,232.993 238.315,232.248 242.447,229.525 242.801,229.283 247.202,226.057 247.287,225.992 251.461,222.59 251.774,222.316 255.296,219.122 
 256.26,218.166 258.747,215.654 260.747,213.405 261.828,212.187 264.3,208.719 265.233,206.798 265.985,205.251 265.605,201.784 265.233,201.496 260.747,201.202 
 258.746,201.784 256.26,202.505 251.774,204.416 250.197,205.251 247.287,206.797 244.038,208.719 242.801,209.464 238.669,212.187 238.315,212.428 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#fabf26; stroke-width:1; stroke-opacity:1; fill:none" points="
 238.315,214.813 237.166,215.654 233.828,218.234 232.739,219.122 229.342,222.094 228.796,222.59 225.468,226.057 224.856,226.853 222.871,229.525 221.041,232.993 
 224.856,235.941 229.342,234.527 232.799,232.993 233.828,232.519 238.315,229.947 238.956,229.525 242.801,226.899 243.95,226.057 247.287,223.478 248.377,222.59 
 251.774,219.617 252.32,219.122 255.648,215.654 256.26,214.859 258.245,212.187 260.075,208.719 256.26,205.77 251.774,207.185 248.316,208.719 247.287,209.192 
 242.801,211.765 242.16,212.187 238.315,214.813 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#f57d15; stroke-width:1; stroke-opacity:1; fill:none" points="
 238.315,210.413 235.842,212.187 233.828,213.675 231.267,215.654 229.342,217.211 227.047,219.122 224.856,221.062 223.152,222.59 220.369,225.287 219.573,226.057 
 216.374,229.525 215.883,230.153 213.631,232.993 211.396,236.258 211.253,236.461 209.892,239.928 210.22,243.396 211.396,244.305 215.883,244.559 220.369,243.507 
 220.632,243.396 224.856,241.669 228.53,239.928 229.342,239.549 233.828,237.076 234.825,236.461 238.315,234.31 240.292,232.993 242.801,231.299 245.273,229.525 
 247.287,228.037 249.849,226.057 251.774,224.501 254.069,222.59 256.26,220.65 257.964,219.122 260.747,216.425 261.543,215.654 264.742,212.187 265.233,211.559 
 267.485,208.719 269.719,205.454 269.863,205.251 271.224,201.784 270.896,198.316 269.719,197.407 265.233,197.153 260.747,198.205 260.484,198.316 256.26,200.043 
 252.586,201.784 251.774,202.163 247.287,204.636 246.291,205.251 242.801,207.402 240.824,208.719 238.315,210.413 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e65e2d; stroke-width:1; stroke-opacity:1; fill:none" points="
 242.801,205.454 238.315,208.451 237.938,208.719 233.828,211.675 233.148,212.187 229.342,215.128 228.68,215.654 224.856,218.831 224.509,219.122 220.632,222.59 
 220.369,222.844 217.047,226.057 215.883,227.294 213.744,229.525 211.396,232.261 210.747,232.993 208.204,236.461 206.91,238.629 206.097,239.928 204.869,243.396 
 205.268,246.864 206.91,248.133 211.396,248.441 215.883,247.492 217.564,246.864 220.369,245.863 224.856,243.898 225.802,243.396 229.342,241.581 232.229,239.928 
 233.828,239.029 237.985,236.461 238.315,236.258 242.801,233.261 243.178,232.993 247.287,230.036 247.968,229.525 251.774,226.583 252.435,226.057 256.26,222.881 
 256.607,222.59 260.484,219.122 260.747,218.868 264.069,215.654 265.233,214.418 267.371,212.187 269.719,209.451 270.369,208.719 272.912,205.251 274.206,203.083 
 275.018,201.784 276.247,198.316 275.847,194.848 274.206,193.579 269.719,193.271 265.233,194.22 263.552,194.848 260.747,195.848 256.26,197.814 255.313,198.316 
 251.774,200.131 248.887,201.784 247.287,202.683 243.13,205.251 242.801,205.454 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#d24742; stroke-wi

In [1]:
include("hmc.jl")

hmc (generic function with 1 method)

In [3]:
S  = [1/3 1/2; 1/2 1];
iS = [12 -6; -6 4];
mu = [0.5, 0.5]

loglik(x)     = -dot(iS*(x-mu), (x-mu))/2
gradloglik(x) = iS*(mu-x)

x0 = [0.2, 0.2]

samples = hmc(loglik, gradloglik, x0; steps=10000, stepsize=1e-1);

In [4]:
xx = [samples[i][1] for i in 1:length(samples)]
yy = [samples[i][2] for i in 1:length(samples)]
;

MethodError: [91mMethodError: Cannot `convert` an object of type Plots.Surface{Array{Float64,2}} to an object of type Array{Float64,1}
This may have arisen from a call to the constructor Array{Float64,1}(...),
since type constructors fall back to convert methods.[39m